In [1]:
import sys

sys.path.append(f"/home/npanj/personal_works/m5-forecasting-accuracy")

from src.utils.import_downcasting import import_downcasting
from src.utils.plotting import PdfFile
from utils import get_wmape, plotting_sales_forecast, get_wmape_custom_groupby
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import copy

root = os.path.join(
    "/home/npanj/personal_works/m5-forecasting-accuracy/assets/data/data_CA_1"
)

In [2]:
data_ca1 = import_downcasting(os.path.join(root, "data_with_arima_resid.csv"))
score_ca1 = import_downcasting(os.path.join(root, "arima_model_score.csv"))

In [3]:
print("data ca1 list columns")
print(data_ca1.info(verbose=True))

print("--------------------------------------------------")
print("data ca1 score list columns")
print(score_ca1.info())

data_ca1 = data_ca1.set_index("date")
data_ca1["arima_residual"] = data_ca1.arima_residual.astype(np.float64)
data_ca1["event_name_1"] = (
    data_ca1["event_name_1"].cat.add_categories("none").fillna("none")
)
data_ca1["event_type_1"] = (
    data_ca1["event_type_1"].cat.add_categories("none").fillna("none")
)

data ca1 list columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918109 entries, 0 to 5918108
Data columns (total 25 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   id              category      
 2   item_id         category      
 3   dept_id         category      
 4   cat_id          category      
 5   store_id        category      
 6   state_id        category      
 7   date_code       category      
 8   sales           int16         
 9   wm_yr_wk        int16         
 10  weekday         category      
 11  wday            int8          
 12  month           int8          
 13  year            int16         
 14  d               category      
 15  event_name_1    category      
 16  event_type_1    category      
 17  event_name_2    category      
 18  event_type_2    category      
 19  snap_CA         int8          
 20  snap_TX         int8          
 21  snap_WI         int8          
 

In [4]:
wmape = get_wmape(data_ca1, "arima_residual")
score_ca1 = pd.merge(
    left=score_ca1,
    right=wmape[["store_id", "item_id", "wmape"]],
    on=["store_id", "item_id"],
    how="left",
)

score_ca1.sort_values(by="wmape", ascending=False, inplace=True)
score_ca1["rank_pct_wmape"] = score_ca1["wmape"].rank(pct=True)
score_ca1 = score_ca1.reset_index(drop=True)

data_ca1["root_square_resid"] = data_ca1["arima_residual"].apply(
    lambda x: math.pow(x, 2)
)

In [5]:
item_id_cat_unq = data_ca1.groupby(["item_id", "cat_id"]).store_id.count().reset_index()
score_ca1 = pd.merge(
    left=score_ca1,
    right=item_id_cat_unq[["item_id", "cat_id"]],
    left_on=["item_id"],
    right_on=["item_id"],
    how="left",
)

score_ca1.info()
##########

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9147 entries, 0 to 9146
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   store_id        9147 non-null   category
 1   item_id         9147 non-null   category
 2   mse             9147 non-null   float16 
 3   mae             9147 non-null   float16 
 4   sse             9147 non-null   float32 
 5   wmape           9147 non-null   float64 
 6   rank_pct_wmape  9147 non-null   float64 
 7   cat_id          9147 non-null   category
dtypes: category(3), float16(2), float32(1), float64(2)
memory usage: 410.2 KB


In [6]:
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_store_ca1.pdf")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_hobbies_ca1.pdf", product_cat="HOBBIES")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_foods_ca1.pdf", product_cat="FOODS")
# plotting_sales_forecast(data=data_ca1, score=score_ca1, file_name="plotting_household_ca1.pdf",\
#                         product_cat="HOUSEHOLD")

# Analysis WMAPE on event_name_type1

- event_name_1 = "Christmas" has sales equal to 0 for all observastion


In [7]:
wmape_by_event_name_1 = get_wmape_custom_groupby(data_ca1, "arima_residual", ["event_name_1"])
wmape_by_event_name_1

,event_name_1,sum_abs_residual,mean_abs_residual,num_obs,sales,wmape
0,Chanukah End,1.365347e+04,0.895603,15245,18946.0,0.720652
1,Christmas,1.991427e+04,1.306282,15245,0.0,inf
2,Cinco De Mayo,1.365034e+04,0.895398,15245,19310.0,0.706905
3,ColumbusDay,1.450234e+04,0.951285,15245,20999.0,0.690621
4,Easter,1.533087e+04,1.005632,15245,22598.0,0.678417
5,Eid al-Fitr,1.413684e+04,0.927310,15245,21417.0,0.660076
6,EidAlAdha,1.382888e+04,0.907109,15245,19528.0,0.708157
7,Father's day,1.086192e+04,0.890613,12196,17326.0,0.626914
8,Halloween,1.354141e+04,0.888253,15245,16767.0,0.807623
9,IndependenceDay,1.400326e+04,0.918548,15245,19665.0,0.712090


# Analysis residual from ARIMA model in the group last 20 percentile

In [8]:
resid_score_worst_20pct = score_ca1[score_ca1.rank_pct_wmape > 0.8]
item_resid_worst_20pct = list(resid_score_worst_20pct.item_id.unique())

data_worst_20pct = data_ca1[data_ca1.item_id.isin(item_resid_worst_20pct)]
data_worst_20pct.to_csv(os.path.join(root, "data_arima_worst_30pct.csv"))

data_worst_20pct = import_downcasting(os.path.join(root, "data_arima_worst_30pct.csv"))

<span style="color:blue">Explore event_name_1 and event_type_1</span>

Finding from rank in event_name_1 that "christmas" event tend to have the most impact on residual

In [9]:
event_name_1 = (
    data_worst_20pct.groupby(["event_name_1"]).root_square_resid.mean().reset_index()
)
event_name_1.rename(columns={"root_square_resid": "mean_rss"}, inplace=True)
event_name_1.sort_values(by="mean_rss", ascending=False, inplace=True)


event_type_1 = (
    data_worst_20pct.groupby(["event_type_1"]).root_square_resid.mean().reset_index()
)
event_type_1.rename(columns={"root_square_resid": "mean_rss"}, inplace=True)
event_type_1.sort_values(by="mean_rss", ascending=False, inplace=True)

In [10]:
event_name_1.head(10)

,event_name_1,mean_rss
1,Christmas,80.474136
10,LaborDay,35.132126
27,Thanksgiving,26.928791
20,OrthodoxEaster,25.622793
4,Easter,24.203529
3,ColumbusDay,22.735889
5,Eid al-Fitr,22.226715
24,Ramadan starts,21.743202
30,none,21.446226
15,Mother's day,21.347801


In [11]:
<span style="color:blue">Explore event_name_1 and event_type_1</span>

SyntaxError: invalid syntax (483449040.py, line 1)